##### Images

The introduction of convolutional neural networks revolutionized computer vision, and image-based systems have since acquired a new set of capabilities.

 To participate in this revolution, you need
 to be able to load images from common image formats and then transform the data
 into a tensor representation that has the various parts of the image arranged in the
 way that PyTorch expects. 

An image is represented as a collection of scalars arranged in a regular grid, having a height and a width (in pixels). You might have a single scalar per grid point (the
 pixel), which would be represented as a grayscale image, or multiple scalars per grid
 point, which typically represent different colors or different features, such as depth
 from a depth camera. 

You have several ways of encoding numbers into colors. The most common is
 RGB, which defines a color with three numbers that represent the intensity of red,
 green and blue. 

In [18]:
import imageio
img_arr = imageio.imread('C:/Users/Haier/blue_tit.jpeg') 
img_arr.shape

(560, 885, 3)

In [19]:
type(img_arr)

imageio.core.util.Array

###### Note: 
PyTorch
 modules that deal with image data require tensors to be laid out as C x H x W (channels, height, and width, respectively)

 You can use the transpose function to get to an appropriate layout. Given an input
 tensor W x H x C, you get to a proper layout by swapping the first and last channels:


In [20]:
import torch
img = torch.from_numpy(img_arr) 
out = torch.transpose(img, 0, 2) 

In [21]:
out.shape

torch.Size([3, 885, 560])

##### Note: 
This operation doesn’t make a copy of the
 tensor data. Instead, out uses the same underlying storage as img and plays with the size
 and stride information at the tensor level

 This arrangement is convenient because the
 operation is cheap, but (heads up) changing a pixel in img leads to a change in out. 

Following the same strategy that you used
 for earlier data types, to create a data set of multiple images to use as an input for your
 neural networks, you store the images in a batch along the first dimension to obtain a
 N x C x H x W tensor.

##### Note: 
As a more efficient alternative to using stack to build up the tensor, you can preallocate a tensor of appropriate size and fill it with images loaded from a directory which indicates that your batch will consist of 100 RGB images 256 pixels in height
 and 256 pixels in width. 

In [22]:
batch_size = 20 
batch = torch.zeros(100, 3, 256, 256, dtype=torch.uint8)

 Now you can load all images from an input directory and store
 them in the tensor:


In [23]:
import os
data_dir = 'C:/Users/Haier/Dataset/vasell_seg_datset/train/vessel'
filenames = [name for name in os.listdir(data_dir)
             if os.path.splitext(name) == '.jpg'] 
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(filename)   
    batch[i] = torch.transpose(torch.from_numpy(img_arr), 0, 2)


In [24]:
batch.shape

torch.Size([100, 3, 256, 256])

In [25]:
batch[0][0].shape

torch.Size([256, 256])

neural networks exhibit the best
 training performance when input data ranges from roughly 0 to 1 or –1 to 1 (an effect
 of how their building blocks are defined). 

In [26]:
# floating point conversion and normalize
batch = batch.float() 
batch /= 255.

Another possibility is to compute mean and standard deviation of the input data and
 scale it so that the output has zero mean and unit standard deviation across each
 channel

In [27]:
n_channels = batch.shape[1] 
for c in range(n_channels): 
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])   
    batch[:, c] = (batch[:, c] - mean) / std

You can perform several other operations on inputs, including geometric transformations such as rotation, scaling, and cropping. These operations may help with
 training or may be required to make an arbitrary input conform to the input
 requirements of a network, such as the size of the image